Compare different classical runs with and without constraint damping to see how they perfom in terms of Hamiltonian, momentum and reduction constraints violations. 

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots; pythonplot()
using DelimitedFiles
using SpheriCo

In [ ]:
# change accordingly
# give the directory where the data from all the runs are saved
base = "../examples/classical_runs/"

par0 = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp0"
par1 = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp1_k1_0.002_k2_0.0"
par2 = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp1_k1_0.0_k2_0.002"
par3 = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp1_k1_0.02_k2_0.0"
par4 = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp1_k1_0.0_k2_0.02"
par5 = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp1_k1_0.002_k2_0.002"
par6 = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp1_k1_0.02_k2_0.02"

# convention, either 1 or 8pi (numerically). Choose the one of your runs
oMp2 = 25.132741228718345

# directories where the data from the different runs are stores
dir0 = base*par0
dir1 = base*par1
dir2 = base*par2
dir3 = base*par3
dir4 = base*par4
dir5 = base*par5
dir6 = base*par6

# directory where plots are saved, tune it accordingly. Here I set it to the parameters above (in the name) ommiting the damping part
par = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345"
out_dir = "./convergence_plots/"*par*"_constraints/"
if ispath(out_dir)==false
    mkpath(out_dir)
end

In [ ]:
# number of grid points in the simulation
D = 5
Nr = 128*2^D + 3 # the overal course graining

# load the r grid
r = h5read(dir1*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its_0, all_filenames_0) = list_h5_files(dir0*"/data_$(Nr)", prefix="data_");
(its_1, all_filenames_1) = list_h5_files(dir1*"/data_$(Nr)", prefix="data_");
(its_2, all_filenames_2) = list_h5_files(dir2*"/data_$(Nr)", prefix="data_");
(its_3, all_filenames_3) = list_h5_files(dir3*"/data_$(Nr)", prefix="data_");
(its_4, all_filenames_4) = list_h5_files(dir4*"/data_$(Nr)", prefix="data_");
(its_5, all_filenames_5) = list_h5_files(dir5*"/data_$(Nr)", prefix="data_");
(its_6, all_filenames_6) = list_h5_files(dir6*"/data_$(Nr)", prefix="data_");

# print the length of all simulations (timesteps). Not all of them have the same. Compare only common timesteps.
println(length(its_0))
println(length(its_1))
println(length(its_2))
println(length(its_3))
println(length(its_4))
println(length(its_5))
println(length(its_6))

# choose the minimum length; common for all simulations
len = minimum([length(its_0), length(its_1), length(its_2), length(its_3), length(its_4), length(its_5), length(its_6)])
println("len=",len)


In [ ]:
# function to reconstruct from saved data the reduction variable Df := (∂_r f)/f
# where f can be e.g. the lapse, or the metric function B
function reconstruct_Df(r::Array, f::Array)
    dr = r[4] - r[3]
    Df = Dr_FD2(f, dr)./f
    return Df
end

The loop below finds the index in the r-grid for the outermost r point that is causally disconnect to the outer boundary rmax.

In [ ]:
# using simulation 1 that has no damping and is the longest (so that point is further in the domain).
it_1      = its_1[end]
it_str_1  = lpad(it_1, 4, "0")
t_end    =  h5readattr(dir1*"/data_$(Nr)/data_$(it_str_1).h5", "/")["time"]
r[end] - t_end
ri = 1
i = 1
while r[i] != r[end] - t_end
    i += 1
end
ri_max_causal = i-1
r[ri_max_causal] == r[end] - t_end
ri_max_causal
r[ri_max_causal]

In [ ]:
# function that finds the time when the apparent horizon first appears
function find_t_AH(your_dir::String, Nr::Int, its_c::Array)
    j = 1
    it_c = its_c[j]
    it_str_c  = lpad(it_c, 4, "0")

    vc    = h5read(your_dir*"/data_$(Nr)/data_$(it_str_c).h5","v")
    tc    = h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_c).h5", "/")["time"]
    r_c = h5read(your_dir*"/data_$(Nr)/r.h5","r")
    Ac    = vc[:,4]
    Bc    = vc[:,5]
    KBc   = vc[:,9]
    AH    = find_AH(r_c, Bc, Ac, KBc)

    while AH < 0
        it_c = its_c[j]
        it_str_c  = lpad(it_c, 4, "0")
        vc = h5read(your_dir*"/data_$(Nr)/data_$(it_str_c).h5","v")
        tc = h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_c).h5", "/")["time"]
        Ac    = vc[:,4]
        Bc    = vc[:,5]
        KBc   = vc[:,9]
        AH = find_AH(r_c, Bc, Ac, KBc)
        j = j + 1  
    end
    t_AH = tc
    return t_AH
end

t_AH = find_t_AH(dir1, Nr, its_1)
println("t_AH = ",t_AH)

Load data from the different runs:

In [ ]:
# index for the its_# lists; common in all
i = 138
it_0 = its_0[i]
it_str_0  = lpad(it_0, 4, "0")

# v_classic = [Φ, Π, Ψ, A, B, DB, Utld, K, KB, λ, α, Dα, Θ, Zr, f, g, U, V]^T
v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
v0 = h5read(dir0*"/data_$(Nr)/data_$(it_str_0).h5","v")
v1 = h5read(dir1*"/data_$(Nr)/data_$(it_str_0).h5","v")
v2 = h5read(dir2*"/data_$(Nr)/data_$(it_str_0).h5","v")
v3 = h5read(dir3*"/data_$(Nr)/data_$(it_str_0).h5","v")
v4 = h5read(dir4*"/data_$(Nr)/data_$(it_str_0).h5","v")
v5 = h5read(dir5*"/data_$(Nr)/data_$(it_str_0).h5","v")
v6 = h5read(dir6*"/data_$(Nr)/data_$(it_str_0).h5","v")

t0 = h5readattr(dir0*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
t1 = h5readattr(dir1*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
t2 = h5readattr(dir2*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
t3 = h5readattr(dir3*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
t4 = h5readattr(dir4*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
t5 = h5readattr(dir5*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
t6 = h5readattr(dir6*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]

# make sure we're comparing the same timestep
@assert t0 ≈ t1 ≈ t2 ≈ t3 ≈ t4 ≈ t5 ≈ t6
println("t=", t0)

Reconstruct the reduction variables from saved data (rec_... functions) and calculate the Hamiltonian and momentum constraint violations., for the time you chose just above:

In [ ]:
rec_DA0 = reconstruct_Df(r, v0[:,4]);
rec_DB0 = reconstruct_Df(r, v0[:,5]);
rec_Utld0 = @. rec_DA0 - 2.0*rec_DB0 - 4.0*v0[:,5]*v0[:,10]/v0[:,4]
rec_Dα0 = reconstruct_Df(r, v0[:,11]);
Ham0, Mom0 = SpheriCo.classical.constraints(v0, dr, oMp2);

rec_DA1 = reconstruct_Df(r, v1[:,4]);
rec_DB1 = reconstruct_Df(r, v1[:,5]);
rec_Utld1 = @. rec_DA1 - 2.0*rec_DB1 - 4.0*v1[:,5]*v1[:,10]/v1[:,4]
rec_Dα1 = reconstruct_Df(r, v1[:,11]);
Ham1, Mom1 = SpheriCo.classical.constraints(v1, dr, oMp2);

rec_DA2 = reconstruct_Df(r, v2[:,4]);
rec_DB2 = reconstruct_Df(r, v2[:,5]);
rec_Utld2 = @. rec_DA2 - 2.0*rec_DB2 - 4.0*v2[:,5]*v2[:,10]/v2[:,4]
rec_Dα2 = reconstruct_Df(r, v2[:,11]);
Ham2, Mom2 = SpheriCo.classical.constraints(v2, dr, oMp2);

rec_DA3 = reconstruct_Df(r, v3[:,4]);
rec_DB3 = reconstruct_Df(r, v3[:,5]);
rec_Utld3 = @. rec_DA3 - 2.0*rec_DB3 - 4.0*v3[:,5]*v3[:,10]/v3[:,4]
rec_Dα3 = reconstruct_Df(r, v3[:,11]);
Ham3, Mom3 = SpheriCo.classical.constraints(v3, dr, oMp2);

rec_DA4 = reconstruct_Df(r, v4[:,4]);
rec_DB4 = reconstruct_Df(r, v4[:,5]);
rec_Utld4 = @. rec_DA4 - 2.0*rec_DB4 - 4.0*v4[:,5]*v4[:,10]/v4[:,4]
rec_Dα4 = reconstruct_Df(r, v4[:,11]);
Ham4, Mom4 = SpheriCo.classical.constraints(v4, dr, oMp2);

rec_DA5 = reconstruct_Df(r, v5[:,4]);
rec_DB5 = reconstruct_Df(r, v5[:,5]);
rec_Utld5 = @. rec_DA5 - 2.0*rec_DB5 - 4.0*v5[:,5]*v5[:,10]/v5[:,4]
rec_Dα5 = reconstruct_Df(r, v5[:,11]);
Ham5, Mom5 = SpheriCo.classical.constraints(v5, dr, oMp2);

rec_DA6 = reconstruct_Df(r, v6[:,4]);
rec_DB6 = reconstruct_Df(r, v6[:,5]);
rec_Utld6 = @. rec_DA6 - 2.0*rec_DB6 - 4.0*v6[:,5]*v6[:,10]/v6[:,4]
rec_Dα6 = reconstruct_Df(r, v6[:,11]);
Ham6, Mom6 = SpheriCo.classical.constraints(v6, dr, oMp2);

Check if there is an apparent horizon in each dataset:

In [ ]:
AH0 = find_AH(r, v0[:,5], v0[:,4], v0[:,9])
println("r_AH0 = ", AH0)

AH1 = find_AH(r, v1[:,5], v1[:,4], v1[:,9])
println("r_AH1 = ", AH1)

AH2 = find_AH(r, v2[:,5], v2[:,4], v2[:,9])
println("r_AH2 = ", AH2)

AH3 = find_AH(r, v3[:,5], v3[:,4], v3[:,9])
println("r_AH3 = ", AH3)

AH4 = find_AH(r, v4[:,5], v4[:,4], v4[:,9])
println("r_AH4 = ", AH4)

AH5 = find_AH(r, v5[:,5], v5[:,4], v5[:,9])
println("r_AH5 = ", AH5)

AH6 = find_AH(r, v6[:,5], v6[:,4], v6[:,9])
println("r_AH6 = ", AH6)


Plot the Hamiltonian and momentum constraint violations in r, for the chosen time, and save the plots in the out_dir you created earlier.

In [ ]:
# ri and rf are the indices for the r-domain in the plot
ri = 3
rf = ri_max_causal

plot(r[ri:rf], abs.(Ham0)[ri:rf], label=L"|H_0|", linewidth = 3, title = "t=$(t0)")
plot!(r[ri:rf], abs.(Ham1)[ri:rf], label=L"|H_1|", linewidth = 2.8, style=:dash)
plot!(r[ri:rf], abs.(Ham2)[ri:rf], label=L"|H_2|", linewidth = 2.6, style=:dot)
plot!(r[ri:rf], abs.(Ham3)[ri:rf], label=L"|H_3|", linewidth = 2.4, style=:dashdot)
plot!(r[ri:rf], abs.(Ham4)[ri:rf], label=L"|H_4|", linewidth = 2.2, style=:dot, color=:blue)

p1 = plot!([AH0], seriestype="vline", label=L"{AH}", linewidth=1, color = "black", frame = true,
     xlim=(-0.2,r[ri_max_causal]+0),
     xticks=([0,5,10,15],["0","5","10","15"]),
     xlabel="r",
     ytickfont=10, yguidefontsize=10,
     xtickfont=10, xguidefontsize=10,
     yscale=:log10,
     ylim=(5*10^-10,10^4),
     yticks=([10^2,10^-2,10^-6],[L"10^{2}",L"10^{-2}",L"10^{-6}"]),
     legendfontsize=10,
)

plot(r[ri:rf], abs.(Mom0)[ri:rf], label=L"|P_0|", linewidth = 3)
plot!(r[ri:rf], abs.(Mom1)[ri:rf], label=L"|P_1|", linewidth = 2.8, style=:dash)
plot!(r[ri:rf], abs.(Mom2)[ri:rf], label=L"|P_2|", linewidth = 2.6, style=:dot)
plot!(r[ri:rf], abs.(Mom3)[ri:rf], label=L"|P_3|", linewidth = 2.4, style=:dashdot)
plot!(r[ri:rf], abs.(Mom4)[ri:rf], label=L"|P_4|", linewidth = 2.2, style=:dot, color=:blue)

p2 = plot!([AH0], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true,
     xlabel="r", xlim=(-0.2,r[ri_max_causal]+0),#
     xticks=([0,5,10,15],["0","5","10","15"]),
     ytickfont=10, yguidefontsize=10,
     xtickfont=10, xguidefontsize=10,
     yticks=([10^2,10^-2,10^-6],["","",""]),
     yscale=:log10,
     ylim=(5*10^-10,10^4),
     legendfontsize=10
)

plt = plot(p1, p2, layout = grid(1,2), wsize = (900,285))

savefig(plt, out_dir*"/tf_Ham_Mom.pdf")

Plot the absolute value of the difference between the reconstructed reduction variables and the evolved ones (DA is not evolved but it can be created by them without taking any derivatives).

In [ ]:
ri = 3
rf = ri_max_causal

plot(r[ri:rf], abs.(v0[:,7] .+ 2.0*v0[:,6] .+ 4.0*v0[:,5].*v0[:,10]./v0[:,4] .- rec_DA0)[ri:rf],
    label=L"|\epsilon(DA_0)|", linewidth = 3, title = "t=$(t0)",)
plot!(r[ri:rf], abs.(v1[:,7] .+ 2.0*v1[:,6] .+ 4.0*v1[:,5].*v1[:,10]./v1[:,4] .- rec_DA1)[ri:rf],
    label=L"|\epsilon(DA_1)|", linewidth = 2.8, style = :dash)
plot!(r[ri:rf], abs.(v2[:,7] .+ 2.0*v2[:,6] .+ 4.0*v2[:,5].*v2[:,10]./v2[:,4] .- rec_DA2)[ri:rf],
    label=L"|\epsilon(DA_2)|", linewidth = 2.6, style=:dot)
plot!(r[ri:rf], abs.(v3[:,7] .+ 2.0*v3[:,6] .+ 4.0*v3[:,5].*v3[:,10]./v3[:,4] .- rec_DA3)[ri:rf],
    label=L"|\epsilon(DA_3)|", linewidth = 2.4, style=:dashdot)
plot!(r[ri:rf], abs.(v4[:,7] .+ 2.0*v4[:,6] .+ 4.0*v4[:,5].*v4[:,10]./v4[:,4] .- rec_DA4)[ri:rf],
    label=L"|\epsilon(DA_4)|", linewidth = 2.2, style=:dot, color=:blue)

p1 = plot!([AH0], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true, 
     xlim=(-0.2,r[ri_max_causal]+0),
     yscale=:log10,
     ylim=(10^-10,10^3),
     yticks = ([10^2,10^-2,10^-6],[L"10^2",L"10^{-2}",L"10^{-6}"]),
     xticks=([0,5,10,15],["0","5","10","15"]),
     legendfontsize=10,
     xtickfont=10, xguidefontsize=10,
     ytickfont=10, yguidefontsize=10,
     xlabel="r"
)

plot(r[ri:rf], abs.(v0[:,6] .- rec_DB0)[ri:rf], label=L"|\epsilon(DB_0)|", linewidth = 3)
plot!(r[ri:rf], abs.(v1[:,6] .- rec_DB1)[ri:rf], label=L"|\epsilon(DB_1)|", linewidth = 2.8, style=:dash)
plot!(r[ri:rf], abs.(v2[:,6] .- rec_DB2)[ri:rf], label=L"|\epsilon(DB_2)|", linewidth = 2.6, style=:dot)
plot!(r[ri:rf], abs.(v3[:,6] .- rec_DB3)[ri:rf], label=L"|\epsilon(DB_3)|", linewidth = 2.4, style=:dashdot)
plot!(r[ri:rf], abs.(v4[:,6] .- rec_DB4)[ri:rf], label=L"|\epsilon(DB_4)|", linewidth = 2.2, style=:dot, color=:blue)

p2 = plot!([AH0], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true, 
     xlim=(-0.2,r[ri_max_causal]+0),
     yscale=:log10,
     ylim=(10^-10,10^3),
     yticks = ([10^2,10^-2,10^-6],["","",""]),
     xticks=([0,5,10,15],["0","5","10","15"]),
     legendfontsize=10,
     xtickfont=10, xguidefontsize=10,
     ytickfont=10, yguidefontsize=10,
     xlabel="r"
)

plot(r[ri:rf], abs.(v0[:,12] .- rec_Dα0)[ri:rf], label=L"|\epsilon(D\alpha_0)|", linewidth = 3)
plot!(r[ri:rf], abs.(v1[:,12] .- rec_Dα1)[ri:rf], label=L"|\epsilon(D\alpha_1)|", linewidth = 2.8, style=:dash)
plot!(r[ri:rf], abs.(v2[:,12] .- rec_Dα2)[ri:rf], label=L"|\epsilon(D\alpha_2)|", linewidth = 2.6, style=:dot)
plot!(r[ri:rf], abs.(v3[:,12] .- rec_Dα3)[ri:rf], label=L"|\epsilon(D\alpha_3)|", linewidth = 2.4, style=:dashdot)
plot!(r[ri:rf], abs.(v4[:,12] .- rec_Dα4)[ri:rf], label=L"|\epsilon(D\alpha_4)|", linewidth = 2.2, style=:dot, color=:blue)

p3 = plot!([AH0], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true, 
     xlabel="r", xlim=(-0.2,r[ri_max_causal]+0),
     yscale=:log10,
     ylim=(10^-10,10^3),
     yticks = ([10^2,10^-2,10^-6],["","",""]),
     xticks=([0,5,10,15],["0","5","10","15"]),
     legendfontsize=10,
     xtickfont=10, xguidefontsize=10,
     ytickfont=10, yguidefontsize=10,
) 

plt = plot(p1, p2, p3, layout = grid(1,3), wsize = (950,290))

savefig(plt, out_dir*"/tf_reduction_constraints.pdf")


Next you can calculate the L2 norms of this violations and plot them.

In [ ]:
# L2 norm of a function f
function l2_norm(f, dr, ri, rf)
    l2 = sqrt.(sum(dr*f[ri:rf].^2))
    return l2
end

In [ ]:
# do the calculation for len, that is the max common timestep of all runs
it_max = len
# calculate the L2 norm in the r-domain, from r=0 to the outer most r that is causally disconnected from rmax
ri = 3
rf = ri_max_causal

# initiate a list to save timesteps of the calculation
t_list = zeros(it_max)

# initiate lists to store norms
l2_Ham0 = zeros(it_max)
l2_Ham1 = zeros(it_max)
l2_Ham2 = zeros(it_max)
l2_Ham3 = zeros(it_max)
l2_Ham4 = zeros(it_max)
l2_Ham5 = zeros(it_max)
l2_Ham6 = zeros(it_max)

l2_Mom0 = zeros(it_max)
l2_Mom1 = zeros(it_max)
l2_Mom2 = zeros(it_max)
l2_Mom3 = zeros(it_max)
l2_Mom4 = zeros(it_max)
l2_Mom5 = zeros(it_max)
l2_Mom6 = zeros(it_max)

l2_E_DA0 = zeros(it_max)
l2_E_DA1 = zeros(it_max)
l2_E_DA2 = zeros(it_max)
l2_E_DA3 = zeros(it_max)
l2_E_DA4 = zeros(it_max)
l2_E_DA5 = zeros(it_max)
l2_E_DA6 = zeros(it_max)

l2_E_DB0 = zeros(it_max)
l2_E_DB1 = zeros(it_max)
l2_E_DB2 = zeros(it_max)
l2_E_DB3 = zeros(it_max)
l2_E_DB4 = zeros(it_max)
l2_E_DB5 = zeros(it_max)
l2_E_DB6 = zeros(it_max)

l2_E_Utld0 = zeros(it_max)
l2_E_Utld1 = zeros(it_max)
l2_E_Utld2 = zeros(it_max)
l2_E_Utld3 = zeros(it_max)
l2_E_Utld4 = zeros(it_max)
l2_E_Utld5 = zeros(it_max)
l2_E_Utld6 = zeros(it_max)

l2_E_Dα0 = zeros(it_max)
l2_E_Dα1 = zeros(it_max)
l2_E_Dα2 = zeros(it_max)
l2_E_Dα3 = zeros(it_max)
l2_E_Dα4 = zeros(it_max)
l2_E_Dα5 = zeros(it_max)
l2_E_Dα6 = zeros(it_max)

# calculate norms
for i in 1:it_max
    
    it_0 = its_0[i]
    it_str_0  = lpad(it_0, 4, "0")
    
    v0 = h5read(dir0*"/data_$(Nr)/data_$(it_str_0).h5","v")
    v1 = h5read(dir1*"/data_$(Nr)/data_$(it_str_0).h5","v")
    v2 = h5read(dir2*"/data_$(Nr)/data_$(it_str_0).h5","v")
    v3 = h5read(dir3*"/data_$(Nr)/data_$(it_str_0).h5","v")
    v4 = h5read(dir4*"/data_$(Nr)/data_$(it_str_0).h5","v")
    v5 = h5read(dir5*"/data_$(Nr)/data_$(it_str_0).h5","v")
    v6 = h5read(dir6*"/data_$(Nr)/data_$(it_str_0).h5","v")
    
    t0 = h5readattr(dir0*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
    t1 = h5readattr(dir1*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
    t2 = h5readattr(dir2*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
    t3 = h5readattr(dir3*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
    t4 = h5readattr(dir4*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
    t5 = h5readattr(dir5*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
    t6 = h5readattr(dir6*"/data_$(Nr)/data_$(it_str_0).h5", "/")["time"]
    
    # make sure we're comparing the same timestep
    @assert t0 ≈ t1 ≈ t2 ≈ t3 ≈ t4 ≈ t5 ≈ t6 
    t_list[i] = t0

    rec_DA0 = reconstruct_Df(r, v0[:,4]);
    rec_DB0 = reconstruct_Df(r, v0[:,5]);
    rec_Utld0 = @. rec_DA0 - 2.0*rec_DB0 - 4.0*v0[:,5]*v0[:,10]/v0[:,4]
    rec_Dα0 = reconstruct_Df(r, v0[:,11]);
    Ham0, Mom0 = SpheriCo.classical.constraints(v0, dr, oMp2);

    rec_DA1 = reconstruct_Df(r, v1[:,4]);
    rec_DB1 = reconstruct_Df(r, v1[:,5]);
    rec_Utld1 = @. rec_DA1 - 2.0*rec_DB1 - 4.0*v1[:,5]*v1[:,10]/v1[:,4]
    rec_Dα1 = reconstruct_Df(r, v1[:,11]);
    Ham1, Mom1 = SpheriCo.classical.constraints(v1, dr, oMp2);
    
    rec_DA2 = reconstruct_Df(r, v2[:,4]);
    rec_DB2 = reconstruct_Df(r, v2[:,5]);
    rec_Utld2 = @. rec_DA2 - 2.0*rec_DB2 - 4.0*v2[:,5]*v2[:,10]/v2[:,4]
    rec_Dα2 = reconstruct_Df(r, v2[:,11]);
    Ham2, Mom2 = SpheriCo.classical.constraints(v2, dr, oMp2);
    
    rec_DA3 = reconstruct_Df(r, v3[:,4]);
    rec_DB3 = reconstruct_Df(r, v3[:,5]);
    rec_Utld3 = @. rec_DA3 - 2.0*rec_DB3 - 4.0*v3[:,5]*v3[:,10]/v3[:,4]
    rec_Dα3 = reconstruct_Df(r, v3[:,11]);
    Ham3, Mom3 = SpheriCo.classical.constraints(v3, dr, oMp2);
    
    rec_DA4 = reconstruct_Df(r, v4[:,4]);
    rec_DB4 = reconstruct_Df(r, v4[:,5]);
    rec_Utld4 = @. rec_DA4 - 2.0*rec_DB4 - 4.0*v4[:,5]*v4[:,10]/v4[:,4]
    rec_Dα4 = reconstruct_Df(r, v4[:,11]);
    Ham4, Mom4 = SpheriCo.classical.constraints(v4, dr, oMp2);
    
    rec_DA5 = reconstruct_Df(r, v5[:,4]);
    rec_DB5 = reconstruct_Df(r, v5[:,5]);
    rec_Utld5 = @. rec_DA5 - 2.0*rec_DB5 - 4.0*v5[:,5]*v5[:,10]/v5[:,4]
    rec_Dα5 = reconstruct_Df(r, v5[:,11]);
    Ham5, Mom5 = SpheriCo.classical.constraints(v5, dr, oMp2);
    
    rec_DA6 = reconstruct_Df(r, v6[:,4]);
    rec_DB6 = reconstruct_Df(r, v6[:,5]);
    rec_Utld6 = @. rec_DA6 - 2.0*rec_DB6 - 4.0*v6[:,5]*v6[:,10]/v6[:,4]
    rec_Dα6 = reconstruct_Df(r, v6[:,11]);
    Ham6, Mom6 = SpheriCo.classical.constraints(v6, dr, oMp2);
    
    l2_Ham0[i] = l2_norm(Ham0, dr, ri, rf)
    l2_Ham1[i] = l2_norm(Ham1, dr, ri, rf)
    l2_Ham2[i] = l2_norm(Ham2, dr, ri, rf)
    l2_Ham3[i] = l2_norm(Ham3, dr, ri, rf)
    l2_Ham4[i] = l2_norm(Ham4, dr, ri, rf)
    l2_Ham5[i] = l2_norm(Ham5, dr, ri, rf)
    l2_Ham6[i] = l2_norm(Ham6, dr, ri, rf)

    l2_Mom0[i] = l2_norm(Mom0, dr, ri, rf)
    l2_Mom1[i] = l2_norm(Mom1, dr, ri, rf)
    l2_Mom2[i] = l2_norm(Mom2, dr, ri, rf)
    l2_Mom3[i] = l2_norm(Mom3, dr, ri, rf)
    l2_Mom4[i] = l2_norm(Mom4, dr, ri, rf)
    l2_Mom5[i] = l2_norm(Mom5, dr, ri, rf)
    l2_Mom6[i] = l2_norm(Mom6, dr, ri, rf)
    
    l2_E_DA0[i] = l2_norm(v0[:,7] .+ 2.0*v0[:,6] .+ 4.0*v0[:,5].*v0[:,10]./v0[:,4] .- rec_DA0, dr, ri, rf)
    l2_E_DA1[i] = l2_norm(v1[:,7] .+ 2.0*v1[:,6] .+ 4.0*v1[:,5].*v1[:,10]./v1[:,4] .- rec_DA1, dr, ri, rf)
    l2_E_DA2[i] = l2_norm(v2[:,7] .+ 2.0*v2[:,6] .+ 4.0*v2[:,5].*v2[:,10]./v2[:,4] .- rec_DA2, dr, ri, rf)
    l2_E_DA3[i] = l2_norm(v3[:,7] .+ 2.0*v3[:,6] .+ 4.0*v3[:,5].*v3[:,10]./v3[:,4] .- rec_DA3, dr, ri, rf)
    l2_E_DA4[i] = l2_norm(v4[:,7] .+ 2.0*v4[:,6] .+ 4.0*v4[:,5].*v4[:,10]./v4[:,4] .- rec_DA4, dr, ri, rf)
    l2_E_DA5[i] = l2_norm(v5[:,7] .+ 2.0*v5[:,6] .+ 4.0*v5[:,5].*v5[:,10]./v5[:,4] .- rec_DA5, dr, ri, rf)
    l2_E_DA6[i] = l2_norm(v6[:,7] .+ 2.0*v6[:,6] .+ 4.0*v6[:,5].*v6[:,10]./v6[:,4] .- rec_DA6, dr, ri, rf)
    
    l2_E_DB0[i] = l2_norm(v0[:,6] .- rec_DB0, dr, ri, rf)
    l2_E_DB1[i] = l2_norm(v1[:,6] .- rec_DB1, dr, ri, rf)
    l2_E_DB2[i] = l2_norm(v2[:,6] .- rec_DB2, dr, ri, rf)
    l2_E_DB3[i] = l2_norm(v3[:,6] .- rec_DB3, dr, ri, rf)
    l2_E_DB4[i] = l2_norm(v4[:,6] .- rec_DB4, dr, ri, rf)
    l2_E_DB5[i] = l2_norm(v5[:,6] .- rec_DB5, dr, ri, rf)
    l2_E_DB6[i] = l2_norm(v6[:,6] .- rec_DB6, dr, ri, rf)
    
    l2_E_Utld0[i] = l2_norm(v0[:,7] .- rec_Utld0, dr, ri, rf)
    l2_E_Utld1[i] = l2_norm(v1[:,7] .- rec_Utld1, dr, ri, rf)
    l2_E_Utld2[i] = l2_norm(v2[:,7] .- rec_Utld2, dr, ri, rf)
    l2_E_Utld3[i] = l2_norm(v3[:,7] .- rec_Utld3, dr, ri, rf)
    l2_E_Utld4[i] = l2_norm(v4[:,7] .- rec_Utld4, dr, ri, rf)
    l2_E_Utld5[i] = l2_norm(v5[:,7] .- rec_Utld5, dr, ri, rf)
    l2_E_Utld6[i] = l2_norm(v6[:,7] .- rec_Utld6, dr, ri, rf)

    l2_E_Dα0[i] = l2_norm(v0[:,12] .- rec_Dα0, dr, ri, rf)
    l2_E_Dα1[i] = l2_norm(v1[:,12] .- rec_Dα1, dr, ri, rf)
    l2_E_Dα2[i] = l2_norm(v2[:,12] .- rec_Dα2, dr, ri, rf)
    l2_E_Dα3[i] = l2_norm(v3[:,12] .- rec_Dα3, dr, ri, rf)
    l2_E_Dα4[i] = l2_norm(v4[:,12] .- rec_Dα4, dr, ri, rf)
    l2_E_Dα5[i] = l2_norm(v5[:,12] .- rec_Dα5, dr, ri, rf)
    l2_E_Dα6[i] = l2_norm(v6[:,12] .- rec_Dα6, dr, ri, rf)
       
end

Plot norms

In [ ]:
ti = 2
tf = 138 # tune until what time you want to plot

plot(t_list[ti:tf], l2_Ham0[ti:tf], label=L"||H_0||", linewidth = 3, legend=:bottomright)
plot!(t_list[ti:tf], l2_Ham1[ti:tf], label=L"||H_1||", linewidth = 2.8, style=:dash)
plot!(t_list[ti:tf], l2_Ham2[ti:tf], label=L"||H_2||", linewidth = 2.6, style=:dot)
plot!(t_list[ti:tf], l2_Ham3[ti:tf], label=L"||H_3||", linewidth = 2.4, style=:dashdot)
plot!(t_list[ti:tf], l2_Ham4[ti:tf], label=L"||H_4||", linewidth = 2.2, style=:dot, color=:blue)

p1 = plot!([t_AH], seriestype="vline", label=L"t_{AH}", linewidth=1, color = "black", frame = true,
    xlabel = "time",
     xlim=(0,8+0),
     yscale=:log10,
     ylim=(10^-5,2*1e-2),
     yticks = ([1e-2,10^-3,10^-4],[L"10^{-2}",L"10^{-3}",L"10^{-4}"]),
     xticks=([0,2,4,6,8],["0","2","4","6","8"]),
     legendfontsize=10,
     xtickfont=10, xguidefontsize=10,
     ytickfont=10, yguidefontsize=10,
)

plot(t_list[ti:tf], l2_Mom0[ti:tf], label=L"||P_0||", linewidth = 3, xlabel="time", legend=:bottomright)
plot!(t_list[ti:tf], l2_Mom1[ti:tf], label=L"||P_1||", linewidth = 2.8, style=:dash)
plot!(t_list[ti:tf], l2_Mom2[ti:tf], label=L"||P_2||", linewidth = 2.6, style=:dot)
plot!(t_list[ti:tf], l2_Mom3[ti:tf], label=L"||P_3||", linewidth = 2.4, style=:dashdot)
plot!(t_list[ti:tf], l2_Mom4[ti:tf], label=L"||P_4||", linewidth = 2.2, style=:dot, color=:blue)

p2 = plot!([t_AH], seriestype="vline", label=L"t_{AH}", linewidth=1, color = "black", frame = true,
     xlim=(0,8+0),
     yscale=:log10,
     ylim=(10^-5,2*1e-2),
     yticks = ([1e-2,10^-3,10^-4],["","",""]),
     xticks=([0,2,4,6,8],["0","2","4","6","8"]),
     legendfontsize=10,
     xtickfont=10, xguidefontsize=10,
     ytickfont=10, yguidefontsize=10,
)

plt = plot(p1, p2, layout = grid(1,2), wsize = (900,275))


savefig(plt, out_dir*"/l2_t_Ham_Mom.pdf")


In [ ]:
ti = 2
tf = 138

plot(t_list[ti:tf], l2_E_DA0[ti:tf], label=L"||\epsilon(D_{A\,0})||", linewidth = 3, legend=:bottomright)
plot!(t_list[ti:tf], l2_E_DA1[ti:tf], label=L"||\epsilon(D_{A\,1})||", linewidth = 2.8, style=:dash)
plot!(t_list[ti:tf], l2_E_DA2[ti:tf], label=L"||\epsilon(D_{A\,2})||", linewidth = 2.6, style=:dot)
plot!(t_list[ti:tf], l2_E_DA3[ti:tf], label=L"||\epsilon(D_{A\,3})||", linewidth = 2.4, style=:dashdot)
plot!(t_list[ti:tf], l2_E_DA4[ti:tf], label=L"||\epsilon(D_{A\,4})||", linewidth = 2.2, style=:dot, color=:blue)

p1 = plot!([t_AH], seriestype="vline", label=L"t_{AH}", linewidth=1, color = "black", frame = true,
     xlabel="time",
     xlim=(0,8+0),
     yscale=:log10,
     ylim=(10^-10,10^-3),
     yticks = ([10^-4,10^-6,10^-8],[L"10^{-4}",L"10^{-6}",L"10^{-8}"]),
     xticks=([0,2,4,6,8],["0","2","4","6","8"]),
     legendfontsize=10,
     xtickfont=10, xguidefontsize=10,
     ytickfont=10, yguidefontsize=10,
)

plot(t_list[ti:tf], l2_E_DB0[ti:tf], label=L"||\epsilon(D_{B\,0})||", linewidth = 3, legend=:bottomright)
plot!(t_list[ti:tf], l2_E_DB1[ti:tf], label=L"||\epsilon(D_{B\,1})||", linewidth = 2.8, style=:dash)
plot!(t_list[ti:tf], l2_E_DB2[ti:tf], label=L"||\epsilon(D_{B\,2})||", linewidth = 2.6, style=:dot)
plot!(t_list[ti:tf], l2_E_DB3[ti:tf], label=L"||\epsilon(D_{B\,3})||", linewidth = 2.4, style=:dashdot)
plot!(t_list[ti:tf], l2_E_DB4[ti:tf], label=L"||\epsilon(D_{B\,4})||", linewidth = 2.2, style=:dot, color=:blue)

p2 = plot!([t_AH], seriestype="vline", label=L"t_{AH}", linewidth=1, color = "black", frame = true,
     xlabel="time",
    xlim=(0,8+0),
     yscale=:log10,
     ylim=(10^-10,10^-3),
     yticks = ([10^-4,10^-6,10^-8],["","",""]),
     xticks=([0,2,4,6,8],["0","2","4","6","8"]),
     legendfontsize=10,
     xtickfont=10, xguidefontsize=10,
     ytickfont=10, yguidefontsize=10,
)

plot(t_list[ti:tf], l2_E_Dα0[ti:tf], label=L"||\epsilon(D_{\alpha\,0})||", linewidth = 3, xlabel="time", legend=:bottomright)
plot!(t_list[ti:tf], l2_E_Dα1[ti:tf], label=L"||\epsilon(D_{\alpha\,1})||", linewidth = 2.8, style=:dash)
plot!(t_list[ti:tf], l2_E_Dα2[ti:tf], label=L"||\epsilon(D_{\alpha\,2})||", linewidth = 2.6, style=:dot)
plot!(t_list[ti:tf], l2_E_Dα3[ti:tf], label=L"||\epsilon(D_{\alpha\,3})||", linewidth = 2.4, style=:dashdot)
plot!(t_list[ti:tf], l2_E_Dα4[ti:tf], label=L"||\epsilon(D_{\alpha\,4})||", linewidth = 2.2, style=:dot, color=:blue)

p3 = plot!([t_AH], seriestype="vline", label=L"t_{AH}", linewidth=1, color = "black", frame = true,
     xlim=(0,8+0),
     yscale=:log10,
     ylim=(10^-10,10^-3),
     yticks = ([10^-4,10^-6,10^-8],["","",""]),
     xticks=([0,2,4,6,8],["0","2","4","6","8"]),
     legendfontsize=10,
     xtickfont=10, xguidefontsize=10,
     ytickfont=10, yguidefontsize=10,
)

plt = plot(p1, p2, p3, layout = grid(1,3), wsize = (900,255))


savefig(plt, out_dir*"/l2_t_reduction_constraints.pdf")
